## Clean up
Please make sure to comment the below section if you are planning to use the Knowledge Base that you created above for building your RAG application.
If you only wanted to try out creating the KB using SDK, then please make sure to delete all the resources that were created as you will be incurred cost for storing documents in OSS index.

#### Delete KnowledgeBase - uncomment and delete resources after completing all the notebooks.


In [6]:
import os
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

In [7]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
from retrying import retry
suffix = 594

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
s3_client = boto3.client('s3')
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"
bucket_name = f'bedrock-kb-{s3_suffix}' # replace it with your bucket name.
pp = pprint.PrettyPrinter(indent=2)

In [8]:
import boto3
import time

vector_store_name = f'bedrock-sample-rag-{suffix}'
index_name = f"bedrock-sample-index-{suffix}"
aoss_client = boto3_session.client('opensearchserverless')

In [9]:
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

In [10]:
ds = {'dataSourceId': '6LFFKKCKV9'}
kb = {'knowledgeBaseId': '7BVHEOMVBK'}

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)
host = "wfh400n1e5sbilgouexa.us-west-2.aoss.amazonaws.com"

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

collection_id = "wfh400n1e5sbilgouexa"

In [11]:
bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb['knowledgeBaseId'])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb['knowledgeBaseId'])
oss_client.indices.delete(index=index_name)
aoss_client.delete_collection(id=collection_id)

aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

NameError: name 'access_policy' is not defined

#### Delete role and policies


In [15]:
from utility import delete_iam_role_and_policies
delete_iam_role_and_policies()

NoSuchEntityException: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: The role with name AmazonBedrockExecutionRoleForKnowledgeBase_526 cannot be found.

#### Delete S3 objects

In [16]:
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'ZJPAF9N6X43F5RCB',
  'HostId': 'w5902T3Dqg7djVpJDpqPJ/hoOVSZrpWfuY1qaTREg/06G796xRfOmBoXffQARzWubDXdyzTP48o=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'w5902T3Dqg7djVpJDpqPJ/hoOVSZrpWfuY1qaTREg/06G796xRfOmBoXffQARzWubDXdyzTP48o=',
   'x-amz-request-id': 'ZJPAF9N6X43F5RCB',
   'date': 'Tue, 30 Apr 2024 14:49:30 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}